# YOLOv5 for License Plate Detection

## Dataset: [Car License Plate Detection](https://www.kaggle.com/andrewmvd/car-plate-detection)
![](https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F793761%2Fc15e812b3ab9aad2c0694a2e1f7548e9%2FUntitled.png?generation=1590981584876269&alt=media)

## Repro [YOLOv5](https://github.com/ultralytics/yolov5)

In [1]:
# Clone YOLOv5 code
!git clone https://github.com/rkuo2000/yolov5
%cd yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 12560, done.
remote: Total 12560 (delta 0), reused 0 (delta 0), pack-reused 12560
Receiving objects: 100% (12560/12560), 12.37 MiB | 26.67 MiB/s, done.
Resolving deltas: 100% (8577/8577), done.
/kaggle/working/yolov5


In [2]:
!echo "train: Dataset/train/images" > data/alpr.yaml
!echo "val:   Dataset/train/images" >> data/alpr.yaml

!echo "nc : 1" >> data/alpr.yaml
!echo "names: ['license']" >> data/alpr.yaml

!cat data/alpr.yaml

train: Dataset/train/images
val:   Dataset/train/images
nc : 1
names: ['license']


## Prepare Dataset

In [3]:
import os
import numpy as np
from pathlib import Path
from xml.dom.minidom import parse
from shutil import copyfile

In [4]:
FILE_ROOT = "/kaggle/input/car-plate-detection/"
IMAGE_PATH = FILE_ROOT + "images"  
ANNOTATIONS_PATH = FILE_ROOT + "annotations"

DATA_ROOT = "Dataset/"
DEST_IMAGES_PATH = "train/images"
DEST_LABELS_PATH = "train/labels"

In [5]:
!mkdir -p Dataset/train/labels

In [6]:
# copy images
!mkdir -p Dataset/train
!cp -rf /kaggle/input/car-plate-detection/images Dataset/train

In [7]:
!mkdir -p Dataset/val
!cp -rf /kaggle/input/car-plate-detection/images/Cars1*.png Dataset/val

### Convert COCO Annotations to YOLOv5 Labels

In [8]:
def cord_converter(size, box):
    """
    convert xml annotation to darknet format coordinates
    :param size： [w,h]
    :param box: anchor box coordinates [upper-left x,uppler-left y,lower-right x, lower-right y]
    :return: converted [x,y,w,h]
    """
    x1 = int(box[0])
    y1 = int(box[1])
    x2 = int(box[2])
    y2 = int(box[3])

    dw = np.float32(1. / int(size[0]))
    dh = np.float32(1. / int(size[1]))

    w = x2 - x1
    h = y2 - y1
    x = x1 + (w / 2)
    y = y1 + (h / 2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]
    
def save_file(img_jpg_file_name, size, img_box):
    classes = ['license']
    save_file_name = DATA_ROOT + DEST_LABELS_PATH + '/' + img_jpg_file_name + '.txt'
    print(save_file_name)
    file_path = open(save_file_name, "a+")
    for box in img_box:                  
        #cls_num = classes.index(box[0]) # find class_id
        cls_num = 0
        new_box = cord_converter(size, box[1:]) # convert box coord into YOLO x,y,w,h

        file_path.write(f"{cls_num} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}\n")

    file_path.flush()
    file_path.close()
    
def get_xml_data(file_path, img_xml_file):
    img_path = file_path + '/' + img_xml_file + '.xml'
    print(img_path)

    dom = parse(img_path)
    root = dom.documentElement
    img_name = root.getElementsByTagName("filename")[0].childNodes[0].data
    img_size = root.getElementsByTagName("size")[0]
    objects = root.getElementsByTagName("object")
    img_w = img_size.getElementsByTagName("width")[0].childNodes[0].data
    img_h = img_size.getElementsByTagName("height")[0].childNodes[0].data
    img_c = img_size.getElementsByTagName("depth")[0].childNodes[0].data
    # print("img_name:", img_name)
    # print("image_info:(w,h,c)", img_w, img_h, img_c)
    img_box = []
    for box in objects:
        cls_name = box.getElementsByTagName("name")[0].childNodes[0].data
        x1 = int(box.getElementsByTagName("xmin")[0].childNodes[0].data)
        y1 = int(box.getElementsByTagName("ymin")[0].childNodes[0].data)
        x2 = int(box.getElementsByTagName("xmax")[0].childNodes[0].data)
        y2 = int(box.getElementsByTagName("ymax")[0].childNodes[0].data)
        print("box:(c,xmin,ymin,xmax,ymax)", cls_name, x1, y1, x2, y2)
        img_jpg_file_name = img_xml_file + '.jpg'
        img_box.append([cls_name, x1, y1, x2, y2])
    # print(img_box)
    # test_dataset_box_feature(img_jpg_file_name, img_box)
    save_file(img_xml_file, [img_w, img_h], img_box)

In [9]:
files = os.listdir(ANNOTATIONS_PATH)
for file in files:
    print("file name: ", file)
    file_xml = file.split(".")
    get_xml_data(ANNOTATIONS_PATH, file_xml[0])

file name:  Cars339.xml
/kaggle/input/car-plate-detection/annotations/Cars339.xml
box:(c,xmin,ymin,xmax,ymax) licence 209 135 283 169
Dataset/train/labels/Cars339.txt
file name:  Cars13.xml
/kaggle/input/car-plate-detection/annotations/Cars13.xml
box:(c,xmin,ymin,xmax,ymax) licence 191 147 242 169
Dataset/train/labels/Cars13.txt
file name:  Cars74.xml
/kaggle/input/car-plate-detection/annotations/Cars74.xml
box:(c,xmin,ymin,xmax,ymax) licence 115 115 277 153
Dataset/train/labels/Cars74.txt
file name:  Cars16.xml
/kaggle/input/car-plate-detection/annotations/Cars16.xml
box:(c,xmin,ymin,xmax,ymax) licence 36 175 62 186
Dataset/train/labels/Cars16.txt
file name:  Cars291.xml
/kaggle/input/car-plate-detection/annotations/Cars291.xml
box:(c,xmin,ymin,xmax,ymax) licence 71 205 215 246
Dataset/train/labels/Cars291.txt
file name:  Cars236.xml
/kaggle/input/car-plate-detection/annotations/Cars236.xml
box:(c,xmin,ymin,xmax,ymax) licence 223 106 250 119
Dataset/train/labels/Cars236.txt
file name:

In [10]:
!mkdir -p Dataset/val/labels
!cp -rf Dataset/train/labels/Cars1*.txt Dataset/val/labels

In [11]:
!ls Dataset/train/labels

Cars0.txt    Cars164.txt  Cars23.txt   Cars296.txt  Cars361.txt  Cars427.txt
Cars1.txt    Cars165.txt  Cars230.txt  Cars297.txt  Cars362.txt  Cars428.txt
Cars10.txt   Cars166.txt  Cars231.txt  Cars298.txt  Cars363.txt  Cars429.txt
Cars100.txt  Cars167.txt  Cars232.txt  Cars299.txt  Cars364.txt  Cars43.txt
Cars101.txt  Cars168.txt  Cars233.txt  Cars3.txt    Cars365.txt  Cars430.txt
Cars102.txt  Cars169.txt  Cars234.txt  Cars30.txt   Cars366.txt  Cars431.txt
Cars103.txt  Cars17.txt   Cars235.txt  Cars300.txt  Cars367.txt  Cars432.txt
Cars104.txt  Cars170.txt  Cars236.txt  Cars301.txt  Cars368.txt  Cars44.txt
Cars105.txt  Cars171.txt  Cars237.txt  Cars302.txt  Cars369.txt  Cars45.txt
Cars106.txt  Cars172.txt  Cars238.txt  Cars303.txt  Cars37.txt	 Cars46.txt
Cars107.txt  Cars173.txt  Cars239.txt  Cars304.txt  Cars370.txt  Cars47.txt
Cars108.txt  Cars174.txt  Cars24.txt   Cars305.txt  Cars371.txt  Cars48.txt
Cars109.txt  Cars175.txt  Cars240.txt  Cars306.txt  Cars372.txt  Cars49.txt
Cars11.

## YOLOv5 Training

In [12]:
import wandb
#Secrets에 있는 Code Snippet 복사해서 붙여넣기
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("wandb")
os.environ["WANDB_API_KEY"] = secret_value
wandb.login()

wandb: Currently logged in as: zzang50aa (use `wandb login --relogin` to force relogin)


True

In [13]:
!python train.py --img 416 --batch 16 --epochs 300 --data data/alpr.yaml --cfg models/yolov5s.yaml --name car_number_plate_yolos_result

train: weights=yolov5s.pt, cfg=models/yolov5s.yaml, data=data/alpr.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=car_number_plate_yolos_result, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/rkuo2000/yolov5 ✅
YOLOv5 🚀 v6.1-303-gb3ca8d8 Python-3.7.10 torch-1.7.0 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0

## YOLOv5 Detect

In [14]:
# Download OpenALPR Benchmarks
%cd ..
!git clone https://github.com/openalpr/benchmarks

/kaggle/working
Cloning into 'benchmarks'...
remote: Enumerating objects: 1752, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 1752 (delta 22), reused 22 (delta 22), pack-reused 1728
Receiving objects: 100% (1752/1752), 187.98 MiB | 54.45 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [15]:
%cd yolov5

/kaggle/working/yolov5


In [16]:
!python detect.py --source ../benchmarks/endtoend/us --conf 0.4 --weights runs/train/exp/weights/best.pt --save-txt

detect: weights=['runs/train/exp/weights/best.pt'], source=../benchmarks/endtoend/us, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-303-gb3ca8d8 Python-3.7.10 torch-1.7.0 CUDA:0 (Tesla T4, 15102MiB)

Traceback (most recent call last):
  File "detect.py", line 256, in <module>
    main(opt)
  File "detect.py", line 251, in main
    run(**vars(opt))
  File "/opt/conda/lib/python3.7/site-packages/torch/autograd/grad_mode.py", line 26, in decorate_context
    return func(*args, **kwargs)
  File "detect.py", line 92, in run
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
  File "/kaggle/working/yolov5/models/comm

In [17]:
!python detect.py --source ../benchmarks/endtoend/eu --conf 0.4 --weights runs/train/exp/weights/best.pt --save-txt

detect: weights=['runs/train/exp/weights/best.pt'], source=../benchmarks/endtoend/eu, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-303-gb3ca8d8 Python-3.7.10 torch-1.7.0 CUDA:0 (Tesla T4, 15102MiB)

Traceback (most recent call last):
  File "detect.py", line 256, in <module>
    main(opt)
  File "detect.py", line 251, in main
    run(**vars(opt))
  File "/opt/conda/lib/python3.7/site-packages/torch/autograd/grad_mode.py", line 26, in decorate_context
    return func(*args, **kwargs)
  File "detect.py", line 92, in run
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
  File "/kaggle/working/yolov5/models/comm

In [18]:
!python detect.py --source ../benchmarks/endtoend/br --conf 0.4 --weights runs/train/exp/weights/best.pt --save-txt

detect: weights=['runs/train/exp/weights/best.pt'], source=../benchmarks/endtoend/br, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-303-gb3ca8d8 Python-3.7.10 torch-1.7.0 CUDA:0 (Tesla T4, 15102MiB)

Traceback (most recent call last):
  File "detect.py", line 256, in <module>
    main(opt)
  File "detect.py", line 251, in main
    run(**vars(opt))
  File "/opt/conda/lib/python3.7/site-packages/torch/autograd/grad_mode.py", line 26, in decorate_context
    return func(*args, **kwargs)
  File "detect.py", line 92, in run
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
  File "/kaggle/working/yolov5/models/comm

### Display Detected Images

In [19]:
from IPython.display import Image

In [20]:
Image('runs/detect/exp/0b86cecf-67d1-4fc0-87c9-b36b0ee228bb.jpg')

FileNotFoundError: No such file or directory: 'runs/detect/exp/0b86cecf-67d1-4fc0-87c9-b36b0ee228bb.jpg'

FileNotFoundError: No such file or directory: 'runs/detect/exp/0b86cecf-67d1-4fc0-87c9-b36b0ee228bb.jpg'

<IPython.core.display.Image object>

In [21]:
Image('runs/detect/exp2/test_001.jpg')

FileNotFoundError: No such file or directory: 'runs/detect/exp2/test_001.jpg'

FileNotFoundError: No such file or directory: 'runs/detect/exp2/test_001.jpg'

<IPython.core.display.Image object>

In [22]:
Image('runs/detect/exp3/PWF3266.jpg')

FileNotFoundError: No such file or directory: 'runs/detect/exp3/PWF3266.jpg'

FileNotFoundError: No such file or directory: 'runs/detect/exp3/PWF3266.jpg'

<IPython.core.display.Image object>

## OCR 

### get detected ALPR bounding box

In [23]:
# list saved .txt
!ls runs/detect/exp/labels

In [24]:
# read .txt to get x,y,w,h of ALPR
def read_txt(filepath):
    f = open(filepath, 'r')
    lines = f.readlines()
         
    # read objects from each line of .txt
    objects = []
    for line in lines:
        line=line.rstrip()
        obj = [int(float(i)) for i in line.split(' ')]
        objects.append(obj)
    #print(objects)
    return objects

### OCR using PyTesseract

In [25]:
!pip install pytesseract

In [26]:
DETECT_PATH = '/kaggle/working/yolov5/runs/detect/exp/'
IMG_NAME    = 'us10'

In [27]:
Image(DETECT_PATH+IMG_NAME+'.jpg')

FileNotFoundError: No such file or directory: '/kaggle/working/yolov5/runs/detect/exp/us10.jpg'

FileNotFoundError: No such file or directory: '/kaggle/working/yolov5/runs/detect/exp/us10.jpg'

<IPython.core.display.Image object>

In [28]:
import pytesseract
import cv2
import matplotlib.pyplot as plt
img = cv2.imread(DETECT_PATH+IMG_NAME+'.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
lics = read_txt(DETECT_PATH +'labels/'+IMG_NAME+'.txt')

for lic in lics:
    c, x, y, w, h = lic
    print(x,y,w,h) # center of the bounding box
    img_alpr = img[y-int(h/2):y+int(h/2),x-int(w/2):x+int(w/2)]
    plt.imshow(img_alpr)
    txt = pytesseract.image_to_string(img_alpr)
    print(txt)
    
#    img = cv2.putText(img, txt, (x-int(w/2),y-int(h/2)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)   
#cv2.imwrite('alpr_us1.jpg', img)

error: OpenCV(4.5.1) /tmp/pip-req-build-tk9iuyva/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
